In [6]:
import sys
import argparse

sys.path.insert(0, "../sd-scripts")
import torch
from library import train_util, sdxl_train_util, sdxl_model_util
from library.sdxl_lpw_stable_diffusion import (
    SdxlStableDiffusionLongPromptWeightingPipeline,
)

from library.video_inpainting_patch import (
    VideoInpaintingPatchPipeline,
    VideoInpaintingPatch,
)

In [3]:
import argparse
import train_network


def setup_parser() -> argparse.ArgumentParser:
    parser = train_network.setup_parser()
    sdxl_train_util.add_sdxl_training_arguments(parser)
    return parser


parser = setup_parser()

argv = [
    "--config_file",
    "/home/longc/data/code/lora-scripts/config/video-example/video-debug.toml",
]
args = parser.parse_args(argv)
args = train_util.read_config_from_file(args, parser, argv)

Loading settings from /home/longc/data/code/lora-scripts/config/video-example/video-debug.toml...
/home/longc/data/code/lora-scripts/config/video-example/video-debug


In [5]:
tokenizer = sdxl_train_util.load_tokenizer(args)
tokenizers = tokenizer if isinstance(tokenizer, list) else [tokenizer]

prepare tokenizer
update token length: 255


In [ ]:
print("preparing accelerator")
accelerator = train_util.prepare_accelerator(args)
is_main_process = accelerator.is_main_process

# mixed precisionに対応した型を用意しておき適宜castする
weight_dtype, save_dtype = train_util.prepare_dtype(args)
vae_dtype = torch.float32 if args.no_half_vae else weight_dtype

In [ ]:
(
    load_stable_diffusion_format,
    text_encoder1,
    text_encoder2,
    vae,
    unet,
    logit_scale,
    ckpt_info,
) = sdxl_train_util.load_target_model(
    args, accelerator, sdxl_model_util.MODEL_VERSION_SDXL_BASE_V1_0, weight_dtype
)

In [ ]:
# load inpainting head
inpainting_head = VideoInpaintingPatch(sdxl_model_util.vae_scale_factor).to(accelerator.device)

In [ ]:
sdxl_train_util.sample_images(accelerator, args, epoch, global_step, device, vae, tokenizer, text_encoder, unet, inpainting_head=inpainting_head)